## Using x and y directions

In [6]:
import numpy as np
import cv2

# Define the parameters
dt = 0.1  # Time step
kernel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
kernel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

gamma = 0.42

# Pad the kernel to match the size of the image
padding = (kernel_x.shape[0] - 1) // 2
kerx_pad = np.pad(kernel_x, padding, 'constant')
kery_pad = np.pad(kernel_y, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s)
def g(s):
    return 1 / (1 + (s**2 + lambda_value**2))

# Function to apply the update rule
def denoise_image(image, less, kerx, kery):
    k_u_x = cv2.filter2D(image, -1, kerx)
    phi_x = g(k_u_x)#k_u_x*g(k_u_x)
    p_a = cv2.filter2D(phi_x, -1, kerx)
    p_x = dt * p_a
    
    k_u_y = cv2.filter2D(image, -1, kery)
    phi_y = g(k_u_y)#k_u_y*g(k_u_y)
    p_b = cv2.filter2D(phi_y, -1, kery)
    p_y = dt * p_b
    
    ans = (((2 + gamma)*image) - less + p_x + p_y)/(1+gamma)
    
#     k_u_y = cv2.filter2D(image, -1, kery)
#     phi_y = k_u_y*g(k_u_y)
#     p_y = dt * phi_y
#     ans_y = (((2 + gamma)*image) - less + p_y)/(1+gamma)
    
#     ans = (ans_x + ans_y)/2.0
    return ans

# Load the image
image = cv2.imread('lll_noisy.png', 0)
image = image.astype(np.float32) / 255.0

denoised_image = image.copy()
img_less = image.copy()
for _ in range(20):
    temp = denoised_image.copy()
    denoised_image = denoise_image(denoised_image, img_less, kerx_pad, kery_pad)
    img_less = temp.copy()

cv2.imwrite('changed_model.jpg', denoised_image * 255.0)

True

### GLI

In [9]:
import numpy as np
import cv2

# Define the parameters
dt = 0.1  # Time step
kernel_x = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
kernel_y = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])

gamma = 0.42

# Pad the kernel to match the size of the image
padding = (kernel_x.shape[0] - 1) // 2
kerx_pad = np.pad(kernel_x, padding, 'constant')
kery_pad = np.pad(kernel_y, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1
alpha_gli = 1.7

def gray_level(image_matrix):
    return (image_matrix/np.max(image_matrix))**alpha_gli

# Function to compute g(s)
def g(s):
    return 1 / (1 + (s**2 + lambda_value**2))

# Function to apply the update rule
def denoise_image(image, less, kerx, kery):
    k_u_x = cv2.filter2D(image, -1, kerx)
    gli_x = gray_level(np.abs(k_u_x))
    phi_x = gli_x*k_u_x*g(k_u_x)
    p_a = cv2.filter2D(phi_x, -1, kerx)
    p_x = dt * p_a
    
    k_u_y = cv2.filter2D(image, -1, kery)
    gli_y = gray_level(np.abs(k_u_y))
    phi_y = gli_y*k_u_y*g(k_u_y)
    p_b = cv2.filter2D(phi_y, -1, kery)
    p_y = dt * p_b
    
    ans = (((2 + gamma)*image) - less + p_x + p_y)/(1+gamma)
    
#     k_u_y = cv2.filter2D(image, -1, kery)
#     gli_y = gray_level(np.abs(k_u_y))
#     phi_y = gli_y*k_u_y*g(k_u_y)
#     p_b = cv2.filter2D(phi_y, -1, kery)
#     p_y = dt * p_b
#     ans_y = (((2 + gamma)*image) - less + p_y)/(1+gamma)
    
#     ans = (ans_x + ans_y)/2.0
    return ans

# Load the image
image = cv2.imread('lll_noisy.png', 0)
image = image.astype(np.float32) / 255.0

denoised_image = image.copy()
img_less = image.copy()
for _ in range(5):
    temp = denoised_image.copy()
    denoised_image = denoise_image(denoised_image, img_less, kerx_pad, kery_pad)
    img_less = temp.copy()

cv2.imwrite('change_gli.jpg', denoised_image * 255.0)

True

## Only X

In [5]:
import numpy as np
import cv2

# Define the parameters
dt = 0.1  # Time step
kernel = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])

gamma = 0.42

# Pad the kernel to match the size of the image
padding = (kernel.shape[0] - 1) // 2
kernel_padded = np.pad(kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s)
def g(s):
    return 1 / (1 + (s**2 + lambda_value**2))

# Function to apply the update rule
def denoise_image(image, less, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u*g(k_u)
    p = dt * phi
    ans = (((2 + gamma)*image) - less + p)/(1+gamma)
    return ans

# Load the image
image = cv2.imread('lena_noisy.png', 0)
image = image.astype(np.float32) / 255.0

denoised_image = image.copy()
img_less = image.copy()
for _ in range(20):
    temp = denoised_image.copy()
    denoised_image = denoise_image(denoised_image, img_less, kernel_padded)
    img_less = temp.copy()

cv2.imwrite('lena_x_perona.jpg', denoised_image * 255.0)

True

### GLI

In [4]:
import numpy as np
import cv2

# Define the parameters
dt = 0.1  # Time step
kernel = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])

gamma = 0.42

# Pad the kernel to match the size of the image
padding = (kernel.shape[0] - 1) // 2
kernel_padded = np.pad(kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1
alpha_gli = 1.7

def gray_level(image_matrix):
    return (image_matrix/np.max(image_matrix))**alpha_gli

# Function to compute g(s)
def g(s):
    return 1 / (1 + (s**2 + lambda_value**2))

# Function to apply the update rule
def denoise_image(image, less, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    gli = gray_level(np.abs(k_u))
    phi = gli*k_u*g(k_u)
    p = dt * phi
    ans = (((2 + gamma)*image) - less + p)/(1+gamma)
    return ans

# Load the image
image = cv2.imread('lena_noisy.png', 0)
image = image.astype(np.float32) / 255.0

denoised_image = image.copy()
img_less = image.copy()
for _ in range(20):
    temp = denoised_image.copy()
    denoised_image = denoise_image(denoised_image, img_less, kernel_padded)
    img_less = temp.copy()

cv2.imwrite('lena_x_gli.jpg', denoised_image * 255.0)

True

## Only Y

In [7]:
import numpy as np
import cv2

# Define the parameters
dt = 0.1  # Time step
kernel = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])

gamma = 0.42

# Pad the kernel to match the size of the image
padding = (kernel.shape[0] - 1) // 2
kernel_padded = np.pad(kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s)
def g(s):
    return 1 / (1 + (s**2 + lambda_value**2))

# Function to apply the update rule
def denoise_image(image, less, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u*g(k_u)
    p = dt * phi
    ans = (((2 + gamma)*image) - less + p)/(1+gamma)
    return ans

# Load the image
image = cv2.imread('lena_noisy.png', 0)
image = image.astype(np.float32) / 255.0

denoised_image = image.copy()
img_less = image.copy()
for _ in range(20):
    temp = denoised_image.copy()
    denoised_image = denoise_image(denoised_image, img_less, kernel_padded)
    img_less = temp.copy()

cv2.imwrite('lena_y_perona.jpg', denoised_image * 255.0)

True

### GLI

In [12]:
import numpy as np
import cv2

# Define the parameters
dt = 0.1  # Time step
kernel = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])

gamma = 0.42

# Pad the kernel to match the size of the image
padding = (kernel.shape[0] - 1) // 2
kernel_padded = np.pad(kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1
alpha_gli = 1.7

def gray_level(image_matrix):
    return (image_matrix/np.max(image_matrix))**alpha_gli

# Function to compute g(s)
def g(s):
    return 1 / (1 + (s**2 + lambda_value**2))

# Function to apply the update rule
def denoise_image(image, less, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    gli = gray_level(np.abs(k_u))
    phi = gli*k_u*g(k_u)
    p = cv2.filter2D(phi, -1, kernel)
    p_y = dt * p
#     phi = gli*k_u*g(k_u)
#     p = dt * phi
    ans = (((2 + gamma)*image) - less + p_y)/(1+gamma)
    return ans

# Load the image
image = cv2.imread('lll_noisy.png', 0)
image = image.astype(np.float32) / 255.0

denoised_image = image.copy()
img_less = image.copy()
for _ in range(5):
    temp = denoised_image.copy()
    denoised_image = denoise_image(denoised_image, img_less, kernel_padded)
    img_less = temp.copy()

cv2.imwrite('changed_gli_y.jpg', denoised_image * 255.0)

True